In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import os ; os.chdir('d:/Git_repos/MARL/simple_referential_game')
from env import ReferentialGame
from agents import Speaker, Listener

In [ ]:
# Define Environment & Agents
vocab_size, num_objects = 5, 3
env = ReferentialGame(vocab_size, num_objects)
speaker = Speaker(vocab_size, num_objects)
listener = Listener(vocab_size, num_objects)

# Optimizer & Loss Function
optimizer = optim.Adam(list(speaker.parameters()) + list(listener.parameters()), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training Loop
num_episodes = 5000
for episode in range(num_episodes):
    target_object = env.reset()  # Pick a random object

    # Encode the target object as a one-hot vector
    speaker_input = torch.eye(num_objects)[target_object].unsqueeze(0)  
    message_probs = speaker(speaker_input)  
    speaker_message = torch.argmax(message_probs, dim=-1)  # Pick the most likely word

    # Convert message into one-hot encoding
    message_one_hot = torch.eye(vocab_size)[speaker_message]  

    # Listener tries to guess the object
    listener_probs = listener(message_one_hot)
    listener_choice = torch.argmax(listener_probs, dim=-1).item()  # Pick most likely object

    # Compute reward based on listener's correctness
    reward = 1 if listener_choice == target_object else 0

    # Compute loss & update models
    loss = criterion(listener_probs, torch.tensor([target_object]))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print Progress
    if episode % 500 == 0:
        print(f"Episode {episode}: Reward = {reward}, Loss = {loss.item()}")
